In [1]:
# Import necessary libraries
## 設定 OpenAI API Key 變數
from dotenv import load_dotenv
import os

# Load the environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv('OPENAI_API_KEY')


## API response by openai GPT x

In [2]:
import requests
import json
from pprint import pprint as pp

def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=300, functions=None, function_call=None, format_type=None):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens }
  if functions:
    payload["functions"] = functions
  if function_call:
    payload["function_call"] = function_call
  if format_type:
    payload["response_format"] =  { "type": format_type }

  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["choices"][0]["message"]
  else :
    return obj["error"]

## long text summarization module

In [4]:

summary_prompt = """ 
# (zh-tw)
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary. If the lines are meaningless just return NONE

EXAMPLE
Current summary:
The human asks who is the lead singer of Motorhead. The AI responds Lemmy Kilmister.

New lines of conversation:
Human: What are the other members of Motorhead?
AI: The original members included Lemmy Kilmister (vocals, bass), Larry Wallis (guitar), and Lucas Fox (drums), with notable members throughout the years including \"Fast\" Eddie Clarke (guitar), Phil \"Philthy Animal\" Taylor (drums), and Mikkey Dee (drums).

New summary:
The human asks who is the lead singer and other members of Motorhead. The AI responds Lemmy Kilmister is the lead singer and other original members include Larry Wallis, and Lucas Fox, with notable past members including \"Fast\" Eddie Clarke, Phil \"Philthy Animal\" Taylor, and Mikkey Dee.
END OF EXAMPLE

Current summary:
{prev_summary}
New lines of conversation:
{messages_joined}
New summary:

"""


In [5]:
# chat completion mod 的 token 數量計算

import tiktoken

# 出自 https://platform.openai.com/docs/guides/gpt/managing-tokens
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
  """Returns the number of tokens used by a list of messages."""
  try:
      encoding = tiktoken.encoding_for_model(model)
  except KeyError:
      encoding = tiktoken.get_encoding("cl100k_base")
  if model == "gpt-3.5-turbo-0613":  # note: future models may deviate from this
      num_tokens = 0
      for message in messages:
          num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
          for key, value in message.items():
              num_tokens += len(encoding.encode(value))
              if key == "name":  # if there's a name, the role is omitted
                  num_tokens += -1  # role is always required and always 1 token
      num_tokens += 2  # every reply is primed with <im_start>assistant
      return num_tokens
  else:
      raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.
  See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")



In [6]:
current_context = None # 這個變數保存目前的對話摘要

def messages_to_string(messages):
    # 只抓 user 和 assistant 的 messages
    messages = filter(lambda m: (m["role"] == 'user' or m["role"] == 'assistant'), messages )

    # 將 messages 轉成 string
    dialogue = []
    for message in messages:
        role = message["role"]
        content = message["content"]

        if role == "user":
            dialogue.append(f"user: {content}")
        elif role == "assistant":
            dialogue.append(f"assistant: {content}")

    return "\n".join(dialogue)

# 當輸入的 messages 超過 max_tokens 時，將所有 user 跟 assistant messages 壓縮成一個 system message
def handle_compaction(messages, max_tokens = 1000):
  
  if num_tokens_from_messages(messages) < max_tokens:
    return messages
  
  else:
    # 當字數超過時，觸發摘要動作
    global current_context
    str = messages_to_string(messages)
    summary_user_prompt = summary_prompt.format(prev_summary = current_context, messages_joined = str)
    response = get_completion( [{"role": "user", "content": summary_user_prompt}], temperature=0)
    current_context = response["content"] ## 更新對話摘要

    # 丟棄 user 和 assistant messages，只保留 system messages
    existing_system_messages = list( filter(lambda m: (m["role"] == 'system'), messages ) )
    # 加入最新的對話摘要 system message
    new_system_messages = [{"role": "system", "content": f"The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n{current_context} """} ]
    
    #　系統對話 + 摘要對話
    return existing_system_messages + new_system_messages
  


## 實作具備問題分類的橋接器模組 (處理用戶非結構化數據)



In [28]:
system_prompt = """

請分類用戶問題，將查詢分類為 primary 類別和 secondary 類別。請以 `JSON` 格式提供輸出，使用以下key: primary 和 secondary

 `JSON` 要用以下格式，最終輸出不能有任何錯誤:
[
  "user request": {
    "primary": "string", // 主要類別
    "secondary": "string", // 次要類別
}]

ps: 以下是公司提供的技術支援類別清單，請確保您的回應符合這個規範。若用戶回答不符合規範，請將之歸類為 un-related。 `
primary 類別有：Billing、Technical Support, Account Management 或 General Inquiry 務必遵守這些指示，以便為用戶提供最佳的支援。

### Billing: 次要類別有 (務必遵守這些分類，以便為用戶提供最佳的支援。)
1. 取消訂閱或升級
2. 添加付款方式
3. 收費解釋
4. 爭議收費
5. 延長服務期限
6. 發票問題處理

### Technical Support: 次要類別有 (務必遵守這些分類，以便為用戶提供最佳的支援。)
1. 故障排除
2. 設備兼容性
3. 軟件更新
4. 網絡連接問題
5. 數據恢復和備份
6. 病毒和惡意軟件清除
 
### Account Management: 次要類別有 (務必遵守這些分類，以便為用戶提供最佳的支援。)
1. 重置密碼
2. 更新個人信息
3. 關閉帳戶
4. 帳戶安全
5. 訂閱管理
6. 帳戶活動記錄查詢

### General Inquiry: 次要類別有 (務必遵守這些分類，以便為用戶提供最佳的支援。)
1. 產品信息
2. 價格
3. 反饋
4. 與人聯絡
5. 促銷和優惠
6. 常見問題解答

### un-related: 次要類別有 (務必遵守這些分類，以便為用戶提供最佳的支援。)
1. 未知問題

"""

In [40]:
import openai
import gradio as gr

desc = "這個 **Router Assistance Bot** 提供客戶關於路由器問題的解決方案。"\
       "它能夠指導客戶進行基本的故障排除，並在必要時將他們轉接至IT支援。"

article = "<h1> 路由器客服支援 </h1>"\
          "<h3>如何使用：</h3> " \
          "<ul><li>在提供的輸入框中描述您的問題。會有專人為你服務</li>" \
          "<li>按照線上客服工程師提供的指示進行檢查和操作。</li>" \
          "<li>若您的問題並非路由器相關問題，我們將轉接其他專員為您服務。</li>" \
          "<li>如果問題無法解決，輸入 'IT support requested' 將您轉接至IT支援。</li></ul>"

categories = []

def handle_compaction(history_openai_format):
    # Compaction logic can be added here if needed
    return history_openai_format

def router_assistance_bot(message, history):
    global categories 
    
    history_openai_format = [
        {"role": "system", 
         "content": system_prompt
        },
    ]
    
    for human, assistant in history:
        history_openai_format.append({"role": "user", "content": human})
        history_openai_format.append({"role": "assistant", "content": assistant})
    
    history_openai_format.append({"role": "user", "content": message})
    
    messages = handle_compaction(history_openai_format)

    response = openai.ChatCompletion.create(
        model='gpt-4-turbo-preview',
        messages=messages,
        temperature=0.1,
        stream=True
    )

    partial_message = ""
    for chunk in response:
        try:
            partial_message += chunk.choices[0].delta.content
        except:
            continue

        yield partial_message

# Close all previous Gradio interfaces
gr.close_all()

# Launch the Gradio Chat Interface
gr.ChatInterface(router_assistance_bot, 
                 theme="Soft",   
                 description="Assistance bot for routing requests",
                 title="Router Assistance Bot"
                ).queue().launch(debug=True)



Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.48.0, however version 4.29.0 is available, please upgrade.
--------
Keyboard interruption in main thread... closing server.


有辦法做到問題分類的模型不多，只靠 prompt 工程還是有可能輸出不穩定，必須仰賴模型的理解力，
後續會介紹 function call 功能，確保輸出是 JSON 

**先透過問題分類再轉外部工具或知識庫的效果可能比較好**